# Choosing the most convenient Neighbourhood of Bengaluru

## Introduction/Business Problem

One of the major problem faced by people when they are thinking of switching cities is choosing the neighbourhood to stay/live in that particular city. There are couple of bigger factors like Comfort, Convenience, and saftey.

Here in this particular notebook we will be analyzing the neighbourhood in Bengaluru City of Karnataka, India to figure out the best neighbourhood in this particular city based on Convenience  of  people.

## Data Description

Here for this report we will be using data set for neighbourhood fetched from foursquare loaction api's. But to get data about Boroughs, PinCode, and Neighbourhood we will scraping this data from a webpage(https://www.indiatvnews.com/pincode/karnataka/bangalore) to create our own dataset.

In [2]:
# installing dependencies
!pip install beautifulsoup4
!pip install geocoder
!pip install folium

# importing dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium

scraping data required from webpage

In [3]:
page = requests.get("https://www.indiatvnews.com/pincode/karnataka/bangalore")
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='alt')
table_rows = table.find_all('tr')

Converting scraped data into dataframe

In [4]:
# this array will hold the table data
temp = []

# adding invidual subarrays for each table array
for tr in table_rows:
    td = tr.find_all('td')
    row = [d.text.strip() for d in td]
    
    if row and row[1] != "NA":
        temp.append(row)

converting array with stored data into dataframe

In [5]:
# creating dataframe out of mentioned array
df = pd.DataFrame(data=temp, columns=['Neighbourhood', 'Borough', 'District', 'State', 'Pincode'])
df = df.drop(['District', 'State'], axis=1)
df = df.iloc[1:]
print(df.shape)
print(df)

(259, 3)
            Neighbourhood          Borough Pincode
1                   Agram  Bangalore South  560007
2      Air Force Hospital  Bangalore North  560007
3            Amruthahalli  Bangalore North  560092
4    Anandnagar Bangalore  Bangalore North  560024
5          Arabic College  Bangalore North  560045
..                    ...              ...     ...
255  Tavarekere Bangalore   Bangaloresouth  562130
256   Thammanayakanahalli           Anekal  562106
257         Vanakanahalli           Anekal  562106
258           Vidyanagara         Bg North  562157
259         Yadavanahalli           Anekal  562107

[259 rows x 3 columns]


As certain Boroughs are misspelled so we will be correcting it in next step and then we will be choosing only 3 Boroughs to work on, i.e Bangalore South and North. Other Borough lies in outskirts of District

In [10]:
df['Borough'] = df['Borough'].replace(['Bangalore North', 'Bangalore north', 'Banglorenorth', 'Bg North', 'Bgnorth'], 'Bangalore North')

df['Borough'] = df['Borough'].replace(['Bangalore South', 'Bangaloresouth', 'Bg South', 'Bgsouth', 'Nla & Bgsouth'], 'Bangalore South')

df['Borough'] = df['Borough'].replace(['Bangalore', 'Banglore'], 'Bangalore')

df = df[df['Borough'].isin(["Bangalore South", "Bangalore North"])]

df_dummy = df

print("Below show dataframe will be used for further research")
print(df)

Below show dataframe will be used for further research
            Neighbourhood          Borough Pincode
1                   Agram  Bangalore South  560007
2      Air Force Hospital  Bangalore North  560007
3            Amruthahalli  Bangalore North  560092
4    Anandnagar Bangalore  Bangalore North  560024
5          Arabic College  Bangalore North  560045
..                    ...              ...     ...
244                Kannur  Bangalore North  562149
246     Madanayakanahalli  Bangalore North  562162
254            Tarahunise  Bangalore North  562157
255  Tavarekere Bangalore  Bangalore South  562130
258           Vidyanagara  Bangalore North  562157

[220 rows x 3 columns]
